In [1]:
pip install openai-whisper --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 47.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.4/156.4 MB 249.4 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803404 sha256=32085a861c635330c5d9de85ebaf87c9a45db315e3cb143fcfce970f44e78811
  Stored in directory: /tmp/pip-ephem-wheel-cache-_bgzw8b2/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper
Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import time
import torch
import whisper
import librosa
import datetime
import requests
import pandas as pd
import numpy as np
import torch.nn as nn
import plotly.graph_objects as go
from collections import Counter

In [3]:
class AudioCNN(nn.Module):
    def __init__(self):
        super(AudioCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(p=0.3)
        self.fc1 = nn.Linear(64 * 32 * 70, 128)
        self.fc2 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.dropout(x)
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

In [4]:
classificatorModel = AudioCNN()
classificatorModel.load_state_dict(torch.load('/kaggle/input/speechmusicclassificator/pytorch/default/2/speech_music_classificator.pth', map_location='cpu', weights_only=True))
classificatorModel.eval()

AudioCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=143360, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [5]:
languageIdentifierModel = whisper.load_model('medium')

100%|█████████████████████████████████████| 1.42G/1.42G [00:35<00:00, 42.7MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [6]:
stream = 'https://stream22.tmwradio.com/tmw.mp3'

In [7]:
def recorder(stream_url):
    with open('piece.mp3', 'wb') as f:
        r = requests.get(stream_url, stream=True)
        for block in r.iter_content(50000):
            f.write(block)
            break
    return 'piece.mp3'

In [8]:
def language_identifier(audio):
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio, n_mels=languageIdentifierModel.dims.n_mels)
    _, probs = languageIdentifierModel.detect_language(mel)
    return max(probs, key=probs.get)

In [9]:
def preprocess(audio, sr=48000):
    mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr)
    log_mel_spectrogram = librosa.power_to_db(mel_spectrogram)
    log_mel_spectrogram = (log_mel_spectrogram - np.mean(log_mel_spectrogram)) / np.std(log_mel_spectrogram)
    return log_mel_spectrogram # (128, 282)

In [10]:
def speech_music_classificator(audio):
    array, sampling_rate = librosa.load(audio, dtype='float32', sr=48000)
    array = preprocess(array[:144000])
    array = torch.tensor(array).unsqueeze(0).unsqueeze(0).float()
    with torch.no_grad():
        output = classificatorModel(array)
    prediction = output.item()
    prediction = 'music' if prediction < 0.5 else 'speech'
    return prediction

In [11]:
LANGUAGES = {'en': 'english', 'zh': 'chinese', 'de': 'german', 'es': 'spanish', 'ru': 'russian', 'ko': 'korean', 'fr': 'french', 'ja': 'japanese', 'pt': 'portuguese', 'tr': 'turkish', 'pl': 'polish', 'ca': 'catalan', 'nl': 'dutch', 'ar': 'arabic', 'sv': 'swedish', 'it': 'italian', 'id': 'indonesian', 'hi': 'hindi', 'fi': 'finnish', 'vi': 'vietnamese', 'he': 'hebrew', 'uk': 'ukrainian', 'el': 'greek', 'ms': 'malay', 'cs': 'czech', 'ro': 'romanian', 'da': 'danish', 'hu': 'hungarian', 'ta': 'tamil', 'no': 'norwegian', 'th': 'thai', 'ur': 'urdu', 'hr': 'croatian', 'bg': 'bulgarian', 'lt': 'lithuanian', 'la': 'latin', 'mi': 'maori', 'ml': 'malayalam', 'cy': 'welsh', 'sk': 'slovak', 'te': 'telugu', 'fa': 'persian', 'lv': 'latvian', 'bn': 'bengali', 'sr': 'serbian', 'az': 'azerbaijani', 'sl': 'slovenian', 'kn': 'kannada', 'et': 'estonian', 'mk': 'macedonian', 'br': 'breton', 'eu': 'basque', 'is': 'icelandic', 'hy': 'armenian', 'ne': 'nepali', 'mn': 'mongolian', 'bs': 'bosnian', 'kk': 'kazakh', 'sq': 'albanian', 'sw': 'swahili', 'gl': 'galician', 'mr': 'marathi', 'pa': 'punjabi', 'si': 'sinhala', 'km': 'khmer', 'sn': 'shona', 'yo': 'yoruba', 'so': 'somali', 'af': 'afrikaans', 'oc': 'occitan', 'ka': 'georgian', 'be': 'belarusian', 'tg': 'tajik', 'sd': 'sindhi', 'gu': 'gujarati', 'am': 'amharic', 'yi': 'yiddish', 'lo': 'lao', 'uz': 'uzbek', 'fo': 'faroese', 'ht': 'haitian creole', 'ps': 'pashto', 'tk': 'turkmen', 'nn': 'nynorsk', 'mt': 'maltese', 'sa': 'sanskrit', 'lb': 'luxembourgish', 'my': 'myanmar', 'bo': 'tibetan', 'tl': 'tagalog', 'mg': 'malagasy', 'as': 'assamese', 'tt': 'tatar', 'haw': 'hawaiian', 'ln': 'lingala', 'ha': 'hausa', 'ba': 'bashkir', 'jw': 'javanese', 'su': 'sundanese', 'yue': 'cantonese'}

In [12]:
def pie_plot_drawer(statistics, plotname):
    statistics_counted = {
        'speech': Counter(statistics['speech']),
        'music': Counter(statistics['music'])
    }

    labels = []
    parents = []
    values = []
    ids = []

    for category in ['speech', 'music']:
        total = sum(statistics_counted[category].values())
        labels.append(category)
        ids.append(category)
        parents.append('')
        values.append(total)
    
        for lang, count in statistics_counted[category].items():
            node_id = f"{category}-{lang}"
            labels.append(lang)
            ids.append(node_id)
            parents.append(category)
            values.append(count)

    fig = go.Figure(go.Sunburst(
        labels=labels,
        parents=parents,
        values=values,
        ids=ids,
        branchvalues='total',
        hoverinfo='label+value+percent parent'
    ))
    
    fig.update_layout(
        title=plotname,
        margin=dict(t=40, l=10, r=10, b=10)
    )
    return fig.to_html(full_html=False, include_plotlyjs='cdn')

In [13]:
def bar_plot_drawer(statistics, plotname):
    splits = list(statistics.keys())
    speech_info = []
    music_info = []
    for bar_name, bar in statistics.items():
        speech_info.append(len(bar['speech']))
        music_info.append(len(bar['music']))
    fig = go.Figure(data=[
        go.Bar(
            name = 'Speech',
            x = splits,
            y = speech_info
        ),
        go.Bar(
            name = 'Music',
            x = splits,
            y = music_info
        )
    ])
    
    fig.update_layout(
        barmode='stack',
        title=plotname
    )
                     
    return fig.to_html(full_html=False, include_plotlyjs=False)

In [14]:
html_start = f'''
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="utf-8">
    </head>
    <body>
    
    '''
html_end = f'''
    </body>
    </html>
    '''

In [15]:
def system(stream, hours=2, html_start=html_start, html_end=html_end):
    statistics = {
        'speech': [],
        'music': []
    }
    start_time = time.time()
    time_for_new_bar = start_time + 60*60
    current_bar = f'{str(time.ctime(start_time))}-{str(time.ctime(time_for_new_bar))}'
    statistics_for_bars = {
        current_bar: {
            'speech': [],
            'music': []
        }
    }
    print(f'started at: {time.ctime(start_time + 60*60*3)}')
    finish_time = start_time + 60*60*hours
    while time.time() < finish_time:
        current_time = time.time()
        if current_time >= time_for_new_bar:
            current_bar = f'{str(time.ctime(time_for_new_bar))}-{str(time.ctime(current_time + 60*60))}'
            time_for_new_bar = current_time + 60*60
            statistics_for_bars[current_bar] = {'speech': [], 'music': []}
        audio = recorder(stream)
        language = language_identifier(audio)
        content = speech_music_classificator(audio)
        statistics[content].append(LANGUAGES[language])
        statistics_for_bars[current_bar][content].append(LANGUAGES[language])
    print(f'finished at: {time.ctime(finish_time + 60*60*3)}')
    html_start += pie_plot_drawer(statistics, f'Pie plot for the whole peroid')
    html_start += bar_plot_drawer(statistics_for_bars, f'Bar plot for each hour separately')
    full_html = html_start + html_end
    with open('charts.html', 'w') as html_f:
        html_f.write(full_html)

In [16]:
system(stream)

started at: Wed May 28 17:19:06 2025
finished at: Wed May 28 19:19:06 2025
